# Kaggle - Tabular Playground Series - Jan 2022

## Model 1

# 1. Modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import datetime

# 2. Data

In [2]:
#paths to the Github repo

train_path = 'https://raw.githubusercontent.com/ccollado7/kaggle-tps-jan2022/main/data/raw/train.csv'
test_path = 'https://raw.githubusercontent.com/ccollado7/kaggle-tps-jan2022/main/data/raw/test.csv'
submission_path = 'https://raw.githubusercontent.com/ccollado7/kaggle-tps-jan2022/main/data/raw/sample_submission.csv'

In [3]:
#train dataset

train = pd.read_csv(train_path)
train.head()

,row_id,date,country,store,product,num_sold
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520
2,2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146
3,3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572
4,4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911


In [4]:
#modify date column to datetime

train['date'] = pd.to_datetime(train['date'])

In [5]:
#verify the change in date column

train.dtypes

row_id               int64
date        datetime64[ns]
country             object
store               object
product             object
num_sold             int64
dtype: object

In [6]:
train['year'] = pd.to_datetime(train['date']).dt.year
train['day_year'] = pd.to_datetime(train['date']).dt.dayofyear #Number of day from 1 to 365

train.head()

,row_id,date,country,store,product,num_sold,year,day_year
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329,2015,1
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520,2015,1
2,2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146,2015,1
3,3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572,2015,1
4,4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911,2015,1


In [7]:
#test dataset

test = pd.read_csv(test_path)
test.head()

,row_id,date,country,store,product
0,26298,2019-01-01,Finland,KaggleMart,Kaggle Mug
1,26299,2019-01-01,Finland,KaggleMart,Kaggle Hat
2,26300,2019-01-01,Finland,KaggleMart,Kaggle Sticker
3,26301,2019-01-01,Finland,KaggleRama,Kaggle Mug
4,26302,2019-01-01,Finland,KaggleRama,Kaggle Hat


In [8]:
test['date'] = pd.to_datetime(test['date'])

test['year'] = pd.to_datetime(test['date']).dt.year
test['num_sold'] = 0
test.head()

,row_id,date,country,store,product,year,num_sold
0,26298,2019-01-01,Finland,KaggleMart,Kaggle Mug,2019,0
1,26299,2019-01-01,Finland,KaggleMart,Kaggle Hat,2019,0
2,26300,2019-01-01,Finland,KaggleMart,Kaggle Sticker,2019,0
3,26301,2019-01-01,Finland,KaggleRama,Kaggle Mug,2019,0
4,26302,2019-01-01,Finland,KaggleRama,Kaggle Hat,2019,0


In [9]:
#number of days in test dataset

len(test['date'].unique())

365

In [10]:
#submit

submit = pd.read_csv(submission_path)
submit.head()

,row_id,num_sold
0,26298,100
1,26299,100
2,26300,100
3,26301,100
4,26302,100


In [11]:
#countrys

countrys = train['country'].unique()
countrys

array(['Finland', 'Norway', 'Sweden'], dtype=object)

In [12]:
#stores

stores = train['store'].unique()
stores

array(['KaggleMart', 'KaggleRama'], dtype=object)

In [13]:
#products

products = train['product'].unique()
products

array(['Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker'], dtype=object)

# 3. Model

In [14]:
df = pd.DataFrame(columns = ['row_id', 'num_sold'])
z = 1
for i in countrys:
    for j in stores:
        for k in products:
            filt = (train['country'] == i) & (train['store'] == j) & (train['product'] == k)
            f = train[filt]
            f_1 = test[filt]
            f_1['day_year'] = pd.to_datetime(f_1['date']).dt.dayofyear
            f_1.set_index('day_year',inplace=True)
            f_1['num_sold'] = f.pivot(index='day_year', columns='year', values='num_sold').drop(366).mean(axis=1)
            f_1_sub = f_1[['row_id','num_sold']]
            df = pd.concat([df,f_1_sub])

In [15]:
df.sort_values(by=['row_id'])
df.reset_index(drop=True, inplace=True)
df.to_csv('modelo_1.csv', index=False)